# Regular Genomics Project **RNA Localisation**


---

**Problem definition**:

TODO


### **1.** Data Preparation

Firstly, we import several necessary packages and load in our data:

In [16]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from notes.utils import read_model_file
from sklearn.model_selection import KFold, train_test_split
from keras.callbacks import EarlyStopping
import datetime

In [17]:
# Initializing test/train split


colab = False  #### Set colab flag ####

if colab:
    np.random.seed(3)
    url = 'https://www.dropbox.com/s/hv4uau8q4wwg00k/final_data.csv?dl=1'
    data_org = pd.read_csv(url)
    test_data = data_org.sample(frac=0.1)
    train_data = data_org.drop(test_data.index)
else:
    np.random.seed(3)
    data_org = pd.read_csv('final_data.csv')
    test_data = data_org.sample(frac=0.1)
    train_data = data_org.drop(test_data.index) # TODO: note: we also have to preprocess the test set similary
    # TODO: colab

data_org

,ERM,KDEL,LMA,MITO,NES,NIK,NLS,NUCP,OMM,gene_id,gene_biotype,seq,struct,m6A_5UTR,m6A_CDS,m6A_3UTR
0,57.045409,35.456782,22.008215,12.355106,22.789983,24.241731,16.970436,29.348389,54.916891,ENSG00000000003,protein_coding,ACCTTGTATTAGGTATTTATTTCCACAAAAGTTTGATGCTTACAAC...,"[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...",0,0,1
1,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2.914814,0.244517,ENSG00000000005,protein_coding,TGTGCACAGAAGTTATATACATATATGGGTATATCTATGTAACAAA...,"[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...",0,0,0
2,17.449430,34.151539,29.338431,22.237585,24.682767,43.612551,38.683963,35.678476,25.348560,ENSG00000000419,protein_coding,TACTTTATGCAAAAAAAAATATACATTTATTTATAGGTCTCAATAC...,"[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...",0,0,0
3,3.830180,2.576734,5.737850,0.761343,2.786808,2.784356,3.382682,2.463676,2.819269,ENSG00000000457,protein_coding,TGACTTTCAAACCATTTTAATATTTCAAATATTCCAGAACAATCCC...,"[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...",0,9,5
4,13.132915,8.782925,10.061390,3.012459,8.821250,6.721117,10.827253,8.005113,6.849962,ENSG00000000460,protein_coding,AACCCGCTCGGGTCCCCTTCCACACTGTGGAAGCTTTGTTCTTTCG...,"[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...",0,0,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13805,0.000000,0.000000,0.506809,0.075893,0.050764,0.000000,0.056586,0.000000,0.000000,ENSG00000281883,protein_coding,GGGAAGAAAGGAGCCTGACTCTTATGATGGAATAACCACAAATCAG...,"[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...",0,2,1
13806,0.105452,0.087130,0.171187,0.016101,0.079057,0.669947,0.171672,0.000000,0.254546,ENSG00000282034,protein_coding,GTGTCGGACGGCATGACAGGCAGCAATCCTGTGTCCCCTGCCTCAT...,"[0.37599998712539673, 0.0, 0.07500000298023224...",0,66,4
13807,0.037093,0.444844,0.425368,0.254467,0.323794,1.803249,1.435483,0.249590,0.483645,ENSG00000282827,protein_coding,ATGGCGGGGACCTCCGCGCCAGGCAGCAAGAGGCGGAGCGAGCCCC...,"[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...",0,0,0
13808,0.000000,0.519421,2.119115,0.360450,0.887939,0.345021,1.274465,0.741954,0.460649,ENSG00000282936,protein_coding,CTACTGGAACGCCCCCCTCAATCTAGCCTCCCCCACATAACTCTCT...,"[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...",0,0,2


### **2** |  Initializations

We use as baseline model the [RNATracker](https://github.com/HarveyYan/RNATracker/blob/master/Models/cnn_bilstm_attention.py) model and a CNN model.


In [18]:
max_seq_len = train_data['seq'].apply(lambda x: len(x)).max()
# MODEL
model_path = "model_architectures/cnn_model_first.yaml"

# Path where to save viz
model_architecture_path = "model_architecture_viz/cnn_model_first.png"

# DataLoader
param_dataLoader_train = {
    'padding_length': max_seq_len,
    "batch_size_train": 32,
    "shuffle_batch_train": True,
    "callbacks": [EarlyStopping(monitor='loss', patience=5)]
    }

param_dataLoader_valid = {
    'padding_length': max_seq_len,
    "batch_size_valid": 32,
    "shuffle_batch_valid": False
    }

# KFold
param_KFold = {
    "n_splits": 5,
    "shuffle": False
    }

In [19]:
# Splitting for 5fold

kf = KFold(**param_KFold)
folds = kf.split(train_data)

In [ ]:
# training, only need for the model initialization to change in general
from models.CNN import CNN

VALIDATION_ACCURACY = []
VALIDATION_LOSS = []

# 80/20 split
train_split, valid_split = train_test_split(train_data, random_state=42, test_size=0.2)

model_params = read_model_file(model_path, input_shape=(max_seq_len, 4))

# Model Parameter
param_model = {
    'optimizer': model_params['optimization']['optimizer'],
    'loss': model_params['optimization']['loss'],
    'metrics': model_params['optimization']['metrics'],
    'epochs': model_params['optimization']['epochs']
}

# for i, (train_split, valid_split) in enumerate(folds):
model = CNN(architecure=model_params['model']['architecture'],
            dropouts=model_params['model']['dropouts'],
            pooling=model_params['model']['pooling'],
            conv=model_params['model']['conv_layers'],
            dense=model_params['model']['dense_layers'],
            **param_model )

history = model.fit(train_data=train_split, **param_dataLoader_train)

results = model.evaluate(eval_data=valid_split, **param_dataLoader_valid)
results = dict(zip(model.model.metrics_names, results))

VALIDATION_ACCURACY.append(results['accuracy'])
VALIDATION_LOSS.append(results['loss'])


Epoch 1/10
311/311 [==============================] - 198s 634ms/step - loss: 2.1922 - accuracy: 0.1857
Epoch 2/10
210/311 [===================>..........] - ETA: 1:04 - loss: 2.1639 - accuracy: 0.2271

In [10]:
model.print_model()
import datetime
time_date = datetime.datetime.now().date()
time_hour = datetime.datetime.now().time()

model_output = f"model_outputs/model_{time_date}_{time_hour}.h5"

model.save_model(model_output)

NameError: name 'model' is not defined

In [ ]:
# summarize history for accuracy
def plot_line_graph(data, title, ylabel, xlabel, legend):
    # for i in range(len(data)):
    plt.plot(data[0])
    plt.plot(data[1])
    plt.title(title)
    plt.ylabel(ylabel)
    plt.xlabel(xlabel)
    plt.legend(legend, loc='upper left')
    plt.show()

def scatter_plot(pred, ground_truth):
    pass

plt_data = [history.history['loss']]
plot_line_graph(plt_data, "Loss Graph", 'loss', 'epoch', ['test', 'val'])

plt_data = [history.history['accuracy']]
plot_line_graph(plt_data, "Accuracy Graph", 'accuracy', 'epoch', ['test', 'val'])

In [ ]:
test_result = model.evaluate(test_data, **param_dataLoader_valid)
result = dict(zip(model.model.metrics_names, test_result))
TEST_ACCURACY = result['accuracy']
TEST_LOSS = result['loss']

In [ ]:
VALIDATION_ACCURACY

In [ ]:
VALIDATION_LOSS